# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

## Install `Tensorflow2.0` 

In [1]:
#!!pip uninstall tensorflow
!pip install tensorflow==2.0.0

^C


In [3]:
import tensorflow as tf

## Get Required Files from Drive

In [3]:
#from google.colab import drive
#drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [5]:
#Set your project path 
import os
project_path = os.chdir('C:\\Great_lakes\\AI\\NLP\\Week-6\\Sarcasm Detection-20200306T005501Z-001\\Sarcasm Detection\\Data')

#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [6]:
def parseJson(fname):
    for line in open(fname, 'r'):
        yield eval(line)

In [7]:
data = list(parseJson('./Sarcasm_Headlines_Dataset.json'))

In [8]:
data[0]

{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
 'is_sarcastic': 0}

In [9]:
data[10]

{'article_link': 'https://www.huffingtonpost.com/entry/airline-passengers-tackle-man-who-rushes-cockpit-in-bomb-threat_us_59302e57e4b07572bdbf9460',
 'headline': 'airline passengers tackle man who rushes cockpit in bomb threat',
 'is_sarcastic': 0}

In [10]:
data[3]

{'article_link': 'https://politics.theonion.com/boehner-just-wants-wife-to-listen-not-come-up-with-alt-1819574302',
 'headline': 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
 'is_sarcastic': 1}

In [12]:
# Let us convert list to pandas dataframe
import pandas as pd
DF_news = pd.DataFrame(data)

In [13]:
DF_news.head(3)

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1


In [14]:
DF_news['is_sarcastic'].value_counts()

0    14985
1    11724
Name: is_sarcastic, dtype: int64

## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [15]:
DF_news.drop('article_link' , axis=1, inplace=True)

In [16]:
DF_news.head(3)

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1


In [17]:
DF_news.shape

(26709, 2)

## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [21]:
X=DF_news['headline']
y=DF_news['is_sarcastic']

In [22]:
X[0].split(" ")

['former',
 'versace',
 'store',
 'clerk',
 'sues',
 'over',
 'secret',
 "'black",
 "code'",
 'for',
 'minority',
 'shoppers']

In [23]:
X.head(1)

0    former versace store clerk sues over secret 'b...
Name: headline, dtype: object

In [24]:
text_length=X.apply(lambda X: X.split(" "))

In [25]:
text_length.head(3)

0    [former, versace, store, clerk, sues, over, se...
1    [the, 'roseanne', revival, catches, up, to, ou...
2    [mom, starting, to, fear, son's, web, series, ...
Name: headline, dtype: object

In [26]:
DF_news['len']=DF_news['headline'].apply(lambda X: len(X.split(" ")))

In [27]:
DF_news.head(3)

,headline,is_sarcastic,len
0,former versace store clerk sues over secret 'b...,0,12
1,the 'roseanne' revival catches up to our thorn...,0,14
2,mom starting to fear son's web series closest ...,1,14


In [28]:
DF_news['len'].max()

39

#### Hence 39 is the maximum length of given all headlines

#**## Modelling**

## Import required modules required for modelling.

In [29]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [30]:
max_features = 10000
maxlen = 39
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

In [31]:
import re, pprint
from nltk import word_tokenize

In [32]:
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP

In [33]:
tokens=Tokenizer(num_words=max_features)
tokens.fit_on_texts(DF_news['headline'])

In [34]:
DF_news.head(3)

,headline,is_sarcastic,len
0,former versace store clerk sues over secret 'b...,0,12
1,the 'roseanne' revival catches up to our thorn...,0,14
2,mom starting to fear son's web series closest ...,1,14


# Define X and y for your model.

In [37]:
X = tokens.texts_to_sequences(DF_news['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(DF_news['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 26709
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0  307  678 3336 2297   47  381 2575    5 2576 8433]
Number of Labels:  26709
0


## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [38]:
word_index = tokens.word_index

In [39]:
vocab_size = max(list(word_index.values())) + 1

In [40]:
vocab_size

29657

#**## Word Embedding**

## Get Glove Word Embeddings

In [44]:
print(project_path)

None


In [48]:
project_path=('C:\\Great_lakes\\AI\\NLP\\Week-6\\Sarcasm Detection-20200306T005501Z-001\\Sarcasm Detection\\Data\\')

In [49]:
glove_file = project_path+ "glove.6B.zip"

In [50]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [52]:
EMBEDDING_FILE = './glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE,errors='ignore'):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd

# Create a weight matrix for words in training docs

In [55]:
embedding_matrix = np.zeros((vocab_size, 200))

for word, i in tokens.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

399997

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [0]:
### Embedding layer for hint 
## model.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
### Bidirectional LSTM layer for hint 
## model.add(Bidirectional(LSTM(128, return_sequences = True)))

In [61]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [62]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(vocab_size, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 39, 128)           3796096   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 39, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 4,051,290
Trainable params: 4,051,290
Non-trainable params: 0
_________________________________________________________________
None


# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [68]:
batch_size = 100
epochs = 5
nlp_fit = model.fit(X , y, Batch=100, epoch=5)

TypeError: Unrecognized keyword arguments: {'Batch': 100, 'epoch': 5}